rule-based method

In [33]:
import os
from openai import OpenAI
import os
import zipfile
import json
import re
import msal
import requests
from dotenv import load_dotenv

In [35]:
load_dotenv()

# Azure application client info
# client_id = 'd6912246-965f-4255-ba2b-fc593915f837'
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
tenant_id = os.getenv('TENANT_ID')
# redirect_uri = 'https://login.microsoftonline.com/common/oauth2/nativeclient'

# Get access token
authority = f'https://login.microsoftonline.com/{tenant_id}'
scopes = ['Files.Read', 'User.Read']
app = msal.PublicClientApplication(client_id, authority=authority)

# Request token
result = app.acquire_token_interactive(scopes=scopes)

if "access_token" in result:
    access_token = result["access_token"]
    headers = {'Authorization': f'Bearer {access_token}'}

# Access "modeldb-code-analysis/modeldb-zips"
endpoint = 'https://graph.microsoft.com/v1.0/me/drive/root:/modeldb-code-analysis/modeldb-zips:/children'
response = requests.get(endpoint, headers=headers)
zip_name_id = {}
if response.status_code == 200:
    files_in_subfolder = response.json().get('value', [])
    for file in files_in_subfolder:
        zip_name_id[file['name']] = file['id']
        print(f"File Name: {file['name']} - File ID: {file['id']}")
else:
    print(f"Error: {response.status_code} - {response.text}")

    
#     endpoint = 'https://graph.microsoft.com/v1.0/me/drive/root/children'
#     response = requests.get(endpoint, headers=headers)

#     if response.status_code == 200:
#         files = response.json().get('value', [])
#         folder_id = None
#         for file in files:
#             if file['name'] == 'modeldb-code-analysis' and file.get('folder'):
#                 folder_id = file['id']
#                 print(f"Found folder 'modeldb-code-analysis' with ID: {folder_id}")
#                 break
#         if not folder_id:
#             print("Folder 'modeldb-code-analysis' not found.")
# else:
#     print("Failed to obtain access token")

File Name: 100603.zip - File ID: 0153IT3SXAD2VJTPSQSBHJZPSW2Z3V7UWE
File Name: 101629.zip - File ID: 0153IT3SQMDXPPCZU7MVCJEGVIB5HNOL65
File Name: 102279.zip - File ID: 0153IT3STKCQUSAZDFEFB3ZQGOV23D5CCQ
File Name: 102288.zip - File ID: 0153IT3SQ5Z6DBLC2Q5ZFYRHH6PG2V5FUD
File Name: 102871.zip - File ID: 0153IT3SUCWBE2OAZMMNHZ27YBIIARMBQ7
File Name: 10360.zip - File ID: 0153IT3SRCF5XUXMLTW5DJZFB2N33CR7IQ
File Name: 104623.zip - File ID: 0153IT3SQYAAKUMXRGFNEK5JWYMTZKJLUJ
File Name: 105383.zip - File ID: 0153IT3SR4S23XHBQROVF3L6OAI44XGDGJ
File Name: 105385.zip - File ID: 0153IT3SVGHNI65UB6ZJFJJE2GI4J6HBFR
File Name: 105501.zip - File ID: 0153IT3SRJ6JKY7FNYPRFKI7LCS2O5Y5G3
File Name: 105506.zip - File ID: 0153IT3SVKXQK6MVTF4JGIGELH6AIP4SYK
File Name: 105507.zip - File ID: 0153IT3SS7FBR5Q6E5EFBYBOXVF72IYWQV
File Name: 105528.zip - File ID: 0153IT3SX2FAFPB3YOEZFK5F77JA5WONHI
File Name: 106551.zip - File ID: 0153IT3STYE3LA2MX43FEYSOQJJ7KNC7YK
File Name: 106891.zip - File ID: 0153IT3SQSATX6Y5

In [32]:
len(zip_name_id)

200

In [ ]:
zip_file_path = '/content/drive/MyDrive/group capstone/modeldb_model/100603.zip'
extract_folder = '/content/drive/MyDrive/group capstone/modeldb_model/100603/'
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("completed")


completed


In [ ]:
# Set the path for the JSON file and the output file
json_file_path = '/content/drive/My Drive/group capstone/manual_classifier_rules.json'
output_file_path = '/content/drive/My Drive/group capstone/modeldb_model/match_file/100603_matched_files.txt'

# Load rules from the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    rules = json.load(json_file)

# Convert rules to a list of regex patterns and corresponding replacements
pattern_mapping = {re.compile(pattern): replacement for pattern, replacement in rules.items()}

matched_files = []
# Set the list of acceptable file extensions
acceptable_extensions = ('.py', '.cpp', '.java', '.m', '.txt','.h','.data','.html','.c')#不确定是否要限制文件类型

# Traverse subfolders in the extract_folder directory
for subfolder in os.listdir(extract_folder):
    subfolder_path = os.path.join(extract_folder, subfolder)
    if os.path.isdir(subfolder_path):  # Check if it's a directory
        for root, _, files in os.walk(subfolder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Check if the file extension is acceptable
                if not file.lower().endswith(acceptable_extensions):
                    continue  # Skip the file if the extension is not acceptable

                # Open the file and check its content against the rules
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    for pattern, replacement_list in pattern_mapping.items():
                        if pattern.search(content):  # If any pattern matches the file content
                            matched_files.append(file_path)
                            break  # Stop checking this file if one rule is matched


# Write matched file contents to a new text file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for file_path in matched_files:
        output_file.write(f'=== {file_path} ===\n')  # Write the file path
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            output_file.write(f.read())  # Write the file content
            output_file.write('\n\n')  # Add a newline between files

print(f"Files matching the rules have been saved to {output_file_path}")


Files matching the rules have been saved to /content/drive/My Drive/group capstone/modeldb_model/match_file/100603_matched_files.txt


In [ ]:
! ls '/content/drive/MyDrive/group capstone/modeldb_model/modeldb-zips'

modeldb-model  modeldb-zips


In [ ]:
def process_model(model_name):
    # Set the path for the zip file and extraction folder based on the model name
    zip_file_path = f'/content/drive/MyDrive/group capstone/modeldb_model/modeldb-zips/modeldb-zips/{model_name}.zip'
    extract_folder = f'/content/drive/MyDrive/group capstone/modeldb_model/modeldb-zips/modeldb-model/{model_name}/'
    os.makedirs(extract_folder, exist_ok=True)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)

    print(f"{model_name} extraction completed")

    # Set the path for the JSON rules file and output file
    json_file_path = '/content/drive/My Drive/group capstone/manual_classifier_rules.json'
    output_file_path = f'/content/drive/My Drive/group capstone/modeldb_model/match_file/{model_name}_matched_files.txt'

    # Load rules from the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        rules = json.load(json_file)

    # Convert rules to a dictionary of regex patterns and replacements
    pattern_mapping = {re.compile(pattern): replacement for pattern, replacement in rules.items()}

    matched_files = []
    # Define acceptable file extensions
    acceptable_extensions = ('.py', '.cpp', '.java', '.m', '.txt', '.h', '.data', '.html', '.c')  # Adjust as needed

    # Traverse subfolders and files in the extraction folder
    for subfolder in os.listdir(extract_folder):
        subfolder_path = os.path.join(extract_folder, subfolder)
        if os.path.isdir(subfolder_path):  # Check if it's a directory
            for root, _, files in os.walk(subfolder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    # Check if the file extension is acceptable
                    if not file.lower().endswith(acceptable_extensions):
                        continue  # Skip the file if the extension is not acceptable

                    # Open the file and check its content against the rules
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                        for pattern, replacement_list in pattern_mapping.items():
                            if pattern.search(content):  # If any rule matches the file content
                                matched_files.append(file_path)
                                break  # Stop checking this file if one rule is matched

    # Write matched file contents to a new text file
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for file_path in matched_files:
            output_file.write(f'=== {file_path} ===\n')  # Write the file path
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                output_file.write(f.read())  # Write the file content
                output_file.write('\n\n')  # Add a newline between files

    print(f"Files matching the rules have been saved to {output_file_path}")


In [ ]:
model_name = '93394'
process_model(model_name)

93394 extraction completed
Files matching the rules have been saved to /content/drive/My Drive/group capstone/modeldb_model/match_file/93394_matched_files.txt


In [ ]:
api_key = os.getenv('API_KEY')
organization=os.getenv('ORGANIZATION')
output_file_path='/content/drive/My Drive/group capstone/modeldb_model/match_file/93394_matched_files.txt'

In [ ]:
with open(output_file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()
client = OpenAI(
    api_key=api_key,
    organization=organization
)


chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"Please analyze the following content and provide some metadata about it:\n\n{file_content}"
        }
    ],
)

response_dict = chat_completion.to_dict()
metadata = response_dict["choices"][0]["message"]["content"].strip()
print("Generated metadata:", metadata)

Generated metadata: Metadata:
- Author: Quentin Huys
- Year: 2006
- Title: Fast population coding Neural Computation
- Description: Example code for fast population coding with sparse spike trains in conjunction with a 2006 paper by Quentin Huys, Zemel RS, Natarajan R and Dayan P.
- License: GNU General Public License
- Website: http://www.gatsby.ucl.ac.uk/~qhuys/code.html
- Email: qhuys@gatsby.ucl.ac.uk
- Files:
  - COPYRIGHT.txt
  - getinf.m
  - getspk.m
  - getstim.m
  - LICENSE.txt
  - main.m
  - param.m
  - plots.m
  - psinf.m
  - pspred.m
  - README.txt
  - setup.m
